# A simple Word2Vec model 

This notebook implements a class called `Word2Vec` which can be customized and trained on a given corpus of text. 

The model is described by two matrices $U$ and $V$; the output vector for a given word $\mathbf{x}$ is $\operatorname{softmax}(VU\mathbf{x})$. 

Note: this implementation is not efficient. It emphasizes clarity and is intended as an aide for examining the details of word2vec. For real-world applications, a package like `gensim` should be used instead. 

In [ ]:
import numpy as np
from collections import Counter
import random

In [ ]:
def softmax(v, axis = None):
    """
    Return softmax of vector or matrix v along given axis. 
    
    > softmax([[1,2],[1,5]],axis=0)
    array([[0.5, 0.04742587],
           [0.5, 0.95257413]])
    """
    shifted = v - np.max(v,axis=axis,keepdims=True) # for overflow reasons
    pos = np.exp(shifted)
    return pos / np.sum(pos,axis=axis,keepdims=True)

def mean(v): return sum(v)/len(v)

In [ ]:
class Word2Vec(object):
    
    def __init__(self, vocab_size, hidden_units = 300):
        self.vocab_size = vocab_size
        self.hidden_units = hidden_units
        self.U = np.random.randn(hidden_units, vocab_size)
        self.V = np.random.randn(vocab_size, hidden_units)
        self.context_radius = 2 # of words left/right of center word
        self.worddict = {}
    
    def __repr__(self):
        return f"Word2Vec({self.vocab_size},{self.hidden_units})"        
    
    def predict(self):
        """
        Return the context-word prediction vectors for all words, 
        packaged together into a matrix. 
        """
        return softmax(self.V @ self.U, axis = 1)
    
    def set_corpus(self, corpus):
        """
        Define the dictionary from words to indices
        """
        words_with_counts = Counter(corpus).most_common(self.vocab_size)
        self.worddict # = TODO: WRITE A DICTIONARY COMPREHENSION TO DEFINE THE WORD DICT
        top_words = set(a for (a,b) in words_with_counts)
        self.corpus = [word for word in corpus if word in top_words]
    
    def context_pairs(self, strings = True): 
        """
        Return a list of word-context tuples. Return strings if `strings` is 
        True and use dictionary to convert to indices otherwise. 
        
        Example: 
        
        > model = Word2Vec(1000)
        > corpus = "the quick brown fox jumped over the lazy dog".split(" ")
        > model.context_pairs(corpus)
        [('brown', ['the', 'quick', 'fox', 'jumped']),
         ('fox', ['quick', 'brown', 'jumped', 'over']),
         ('jumped', ['brown', 'fox', 'over', 'the']),
         ('over', ['fox', 'jumped', 'the', 'lazy']),
         ('the', ['jumped', 'over', 'lazy', 'dog'])]
        """
        w = self.context_radius 
        corpus = self.corpus if strings else [self.worddict[word] for word in self.corpus]
        return # TODO
        
    def loss_grad(self, word_index, context_indices, num_negative_samples): 
        """
        Return gradient of loss function with respect to U and V, for a particular 
        word-context pair
        """
        x = np.zeros(self.vocab_size)
        x[word_index] = 1
        y = self.U @ x
        z = self.V @ y 
        inds = context_indices + self.negative_sample(context_indices, num_negative_samples)
        l # TODO: define the vector l, which is the post-softmax vector of length vocab_size
          #       note that l should be zero except in the positions specified by `inds`
        context_vector = np.zeros(self.vocab_size)
        for index in context_indices:
            context_vector[index] = 1
        dLdV = np.outer((-context_vector + l), y.T) # see page 38 of data1010-learning.pdf
        dLdU = np.outer((-context_vector + l) @ self.V, x.T)
        return (dLdU, dLdV) 
        
    def crossentropy(self, context_words, prediction_vector):
        """
        Return loss for a single center-word/context-word pair. 
        
        context_words: list of indices of context words
        prediction_vector: vector of length vocab_size
        """
        return # TODO
    
    def loss(self):
        """
        Return the total loss
        """
        pairs = self.context_pairs(strings = False)
        prediction_matrix = self.predict()
        return # TODO
    
    def negative_sample(self, removed_words, n):
        """
        Choose n words from the worddict other than the words in the list 
        removed_words (which consists of dictionary entries)
        """
        return random.sample(set(self.worddict.values()).difference(removed_words), n)
    
    def train(self, batch_size = 100, num_negative_samples = 100,
              epochs = 1, learning_rate = 0.1):
        context_pairs = self.context_pairs(strings = False)
        num_batches = int(epochs * len(context_pairs) / batch_size)
        for i in range(num_batches):
            print(f"{i}/{num_batches} ",end="")
            inds = np.random.randint(0, len(context_pairs), batch_size)
            ΔUs, ΔVs = # TODO: use loss_grad to find derivatives of loss w.r.t. U and V
            self.U -= learning_rate * mean(ΔUs)
            self.V -= learning_rate * mean(ΔVs)
            print(self.loss())

In [ ]:
def alpha_or_space(s):
    return str.isalpha(s) or s == " "

def sanitize(s):
    return [word for word in "".join(filter(alpha_or_space, s)).lower().split(" ") if word]

with open("mobydick.txt") as f:
    corpus = f.read()

In [ ]:
model = Word2Vec(3000)
model.set_corpus(sanitize("".join(corpus)))
model.loss()

Training is going to take way too long. You can declare victory if you see the training error dropping initially. 

In [ ]:
model.train()